In [ ]:
from src.serial_utilities.serial_utilities import (
    detect_baudrate,
    detect_port,
    test_connection,
    list_ports_info,
    print_ports_info,
    scan_ports,
    TestCase,
)
import logging

logging.basicConfig(level=logging.DEBUG)

print_ports_info()
test_connection("COM3", 115200, test_case=TestCase.default())
scan_ports(test_case=TestCase.default())


In [1]:
from src.stepper.stepper_move import Enable, Disable, Jog, Move, EStop, SyncMove
from src.stepper.stepper_home import SetHome, Home, StopHome, RetrieveHomeParam, SetHomeParam, GetHomeStatus

from src.stepper.stepper_parameters import PositionParams, JogParams, DeviceParams
from src.stepper.stepper_constants import Address, SyncFlag, Direction, Speed, Acceleration, PulseCount, AbsoluteFlag, HomingMode

from serial import Serial
 
# from time import sleep
# import logging
# logging.basicConfig(level=logging.DEBUG)


serial_connection = Serial("COM3", 115200, timeout=0.1)

device_params = DeviceParams(
    serial_connection=serial_connection,
    address=Address(0x01),
)

position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(500),
    acceleration=Acceleration(127),
    pulse_count=PulseCount(160),
    absolute=AbsoluteFlag.RELATIVE,
)

jog_params = JogParams(
    direction=Direction.CW,
    speed=Speed(1),
    acceleration=Acceleration(255),
)

In [ ]:
Enable(device=device_params).status
Move(device=device_params, params=position_params).status


In [ ]:
Disable(device=device_params).status
Move(device=device_params, params=position_params).status

In [ ]:
Jog(device=device_params, params=jog_params).status

In [ ]:
EStop(device=device_params).status

In [ ]:
Move(device=device_params, params=position_params, setting=SyncFlag.SYNC).is_success
SyncMove(device=device_params).status

In [ ]:
position_params = PositionParams(
    direction=Direction.CW,
    speed=Speed(100),
    acceleration=Acceleration(255),
    pulse_count=PulseCount(6400),
    absolute=AbsoluteFlag.RELATIVE,
)

SetHome(device=device_params).status
Move(device=device_params, params=position_params).status



In [ ]:
Home(device=device_params, params=HomingMode.MULTI_TURN_UNLIMITED).status
# sleep(1)
# StopHome(device=device_params).status

In [ ]:
RetrieveHomeParam(device=device_params).data

In [ ]:
homing_params = RetrieveHomeParam(device=device_params).raw_data


In [ ]:
homing_params.bytes

In [ ]:
SetHomeParam(device=device_params, params=homing_params).status

In [ ]:
Move(device=device_params, params=position_params).status



In [ ]:
Home(device=device_params, params=HomingMode.SINGLE_TURN_NEAREST).status

In [ ]:
GetHomeStatus(device=device_params).raw_data.__dict__

In [ ]:
from src.stepper.stepper_sys import CalibrateEncoder

# Unlock system command

calibrate_encoder = CalibrateEncoder
calibrate_encoder.unlock()
calibrate_encoder(device=device_params).status


In [2]:
from src.stepper.stepper_get import (
    GetBusVoltage,
    GetPhaseCurrent,
    GetEncoderValue,
    GetVersion,
    GetMotorRH,
    GetPID,
    GetConfig,
    GetStatus,
    GetError,
    GetSysStatus,
)

print(GetVersion(device=device_params).data)
print(GetMotorRH(device=device_params).data)
print(GetPID(device=device_params).data)
print(GetConfig(device=device_params).data)
print(GetStatus(device=device_params).data)
print(GetError(device=device_params).data)
print(GetSysStatus(device=device_params).data)
print(GetBusVoltage(device=device_params).data)
print(GetPhaseCurrent(device=device_params).data)
print(GetEncoderValue(device=device_params).data)



{'firmware_version': 2, 'hardware_version': 130}
{'phase_resistance mOhm': 2120.0, 'phase_inductance uH': 3180.0}
{'pid_p': 64000, 'pid_i': 100, 'pid_d': 64000}
{'stepper_type': <MotorType.D18: 25>, 'control_mode': <ControlMode.PUL_FOC: 2>, 'communication_mode': <CommunicationMode.UART: 2>, 'enable_level': <EnableLevel.HOLD: 2>, 'default_direction': <Direction.CW: 0>, 'microsteps': 16, 'microstep_interp': <MicrostepInterp.ENABLE: 1>, 'screen_off': <ScreenOff.DISABLE: 0>, 'open_loop_current': 1000, 'max_closed_loop_current': 2400, 'max_voltage': 5000, 'baud_rate': <BaudRate.BAUD_115200: 5>, 'can_rate': <CanRate.CAN_500K: 7>, 'address': 1, 'checksum_mode': <ChecksumMode.FIXED: 0>, 'response_mode': <ResponseMode.RECEIVE: 1>, 'stall_protect': <StallProtect.ENABLE: 1>, 'stall_speed': 8, 'stall_current': 2200, 'stall_time': 2000, 'on_target_window': 3, 'current_unit': <CurrentUnit.mA: 1>, 'voltage_unit': <VoltageUnit.mV: 1>, 'angle_unit': <AngleUnit.deg: 182.04444444444445>, 'time_unit': <Ti

In [ ]:
a = 1
b = bytes(2)*2
len(b)
